In [14]:
import openai
import re
from dotenv import load_dotenv


In [15]:

# Load API credentials
load_dotenv()
from openai import OpenAI
client = OpenAI()

In [17]:
# Agent class that handles interaction
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        chat_completion = client.chat.completions.create(
            model="gpt-4o",
            temperature=0,
            messages=self.messages,
        )
        return chat_completion.choices[0].message.content

In [19]:

# System prompt defining behavior and available actions
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end, output an Answer.

Available actions:

get_flight_time:
e.g. get_flight_time: New York to Paris
Returns average flight time in hours.

calculate:
e.g. calculate: 7.5 + 2.0
Returns the result of the calculation.

Example:
Question: How long will it take if I have a 2 hour layover on a flight from New York to Paris?
Thought: I need the average flight time between New York and Paris.
Action: get_flight_time: New York to Paris
PAUSE

Then, the system will be called again with:
Observation: The flight takes 7.5 hours.

It will then say:
Thought: Now I can calculate total travel time.
Action: calculate: 7.5 + 2.0
PAUSE

Observation: 9.5
Answer: The total travel time is 9.5 hours.
"""

In [20]:

# Action functions
def get_flight_time(query):
    flights = {
        "new york to paris": 7.5,
        "london to tokyo": 11.5,
        "delhi to sydney": 12.0,
    }
    return f"The flight takes {flights.get(query.lower(), 10)} hours."

def calculate(expression):
    try:
        return str(eval(expression))
    except:
        return "Error in calculation."

In [21]:
# Action registry
known_actions = {
    "get_flight_time": get_flight_time,
    "calculate": calculate,
}

# Regex to parse action output from model
action_re = re.compile('^Action: (\w+): (.*)$')

In [22]:

# Main query loop
def query(question, max_turns=5):
    bot = Agent(prompt)
    next_prompt = question
    for _ in range(max_turns):
        result = bot(next_prompt)
        print(result)

        match = [action_re.match(line) for line in result.split('\n') if action_re.match(line)]
        if match:
            action, arg = match[0].groups()
            if action not in known_actions:
                print(f"Unknown action: {action}")
                return
            print(f"-- running {action}({arg})")
            obs = known_actions[action](arg)
            print("Observation:", obs)
            next_prompt = f"Observation: {obs}"
        else:
            break




In [23]:
# Sample run
question = "I have a 2 hour layover on my flight from New York to Paris. What's my total travel time?"
query(question)

Thought: I need the average flight time between New York and Paris.
Action: get_flight_time: New York to Paris
-- running get_flight_time(New York to Paris)
Observation: The flight takes 7.5 hours.
Thought: Now I can calculate total travel time.
Action: calculate: 7.5 + 2.0
-- running calculate(7.5 + 2.0)
Observation: 9.5
Answer: The total travel time, including your 2-hour layover, is 9.5 hours.
